In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
import pandas as pd
import time
from datetime import datetime
import pandas as pd

In [ ]:
#datos del usario. DF columan 0=usuario, columna 1= clave
usuario=pd.read_excel('contrasena_usuario.xlsx')

In [ ]:
#datos de los pacientes y sus practicas. DF columna 0=paciente, columna 1=n afiliado, columna 2=codigo de prestacion
practicas=pd.read_excel('practicas.xlsx')

In [ ]:
#acceso y login
driver = webdriver.Chrome(service=Service(executable_path=ChromeDriverManager().install()))
driver.get("https://cup.pami.org.ar/controllers/loginController.php")
usuario=WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,"//*[@id='usua_logeo']")))
usuario.send_keys(usuario.usuario[0]) #anonimizar
driver.find_element(By.ID, "password").send_keys(usuario.clave[0])#anonimizar
ingresa=WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,"//*[@id='nu_ingresar']/input")))
time.sleep(2)
ingresa.click()
#ingresar al menu ome y abrir la ventana ome, guardar handle para hacer los switch posteriormente
busque_ome=driver.find_elements(By.ID,'cup_ome')
pagina_cup=driver.current_window_handle
#si la ventana esta chica y el menu no se ve
if busque_ome:
    busque_ome[0].click()
else:
    driver.find_element(By.XPATH,"//*[@id='sades_desplegable_box']/label").click()
    ome=WebDriverWait(driver,5).until(EC.visibility_of_element_located((By.ID,'cup_mome')))
    ome.click()
#cambio a pagina OME primera pestana, buscar paciente por numero de afiliado
pagina_ome=driver.window_handles[1]
driver.switch_to.window(pagina_ome)
#loop para cargar todas las practicas por paciente
for i in practicas.index:
    driver.find_element(By.ID, "pe-n_afiliado").send_keys(practicas['naf'][i])
    driver.find_element(By.ID, "pe-btn-bsq-afiliado").click()
    time.sleep(1)
    #cargar codigo diagnostico
    driver.find_element(By.ID, "pe-diagnostico_cod").send_keys("I10")# vamos a dejar siempre el mismo diagnostico para simplicar
    diag=WebDriverWait(driver,5).until(EC.visibility_of_element_located(
        (By.XPATH,"//*[@id='main']/div/section[3]/div/div[1]/div/ul/li")))
    diag.click()
    #cargar prestacion
    driver.find_element(By.ID, "pe-practica").send_keys(practicas['codigo'][i])#aca va el codigo de la practica
    cod_prest=WebDriverWait(driver,5).until(EC.visibility_of_element_located(
        (By.XPATH,"//*[@id='main']/div/section[4]/div[2]/div[1]/div/ul/li/a")))
    cod_prest.click()

#Generar todas las consultas cargadas
driver.find_element(By.ID, "pe-btn-generar-orden").click()
time.sleep(1)
driver.find_element(By.ID, "pe-btn-confirmar-generar-orden").click()
time.sleep(1)
driver.find_element(By.XPATH, "/html/body/div[7]/button").click()
